In [1]:
import scanpy as sc
import pandas as pd

In [2]:
adata = sc.read_h5ad('/Volumes/Castelo_Branco/christoffer/human_ms_xenium/data/macnair/macnair_et_al.h5ad')

/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
adata.write('../../data/MacNair.h5ad')

In [5]:
human_genes = [
    "HIF1A",     # Hypoxia inducible factor 1 alpha
    "HK2",       # Hexokinase 2
    "LDHA",      # Lactate dehydrogenase A
    "LDHB",      # Lactate dehydrogenase B
    "SLC16A1",   # Monocarboxylate transporter 1 (MCT1)
    "SLC16A3",   # Monocarboxylate transporter 4 (MCT4)

        "PDK1",      # Pyruvate dehydrogenase kinase 1
    "PFKL",      # Phosphofructokinase, liver type
   
    "MFN1",      # Mitofusin 1
    "MFN2",      # Mitofusin 2
    "OPA1",      # OPA1 mitochondrial dynamin like GTPase

    "PKM",       # Pyruvate kinase M1/2
    "PPARGC1A",  # PGC-1α, transcriptional coactivator
    "SIRT2",     # Sirtuin 2

]

In [7]:
adata.X.max()

12237.0

In [8]:
# 1) Normalize per cell (counts → CPM/TPM-like)
sc.pp.normalize_total(adata, target_sum=1e4)

# 2) Log-transform
sc.pp.log1p(adata)

In [21]:
list(adata.obs.type_broad.unique())

['Astrocytes',
 'Excitatory neurons',
 'OPCs + COPs',
 'Endo + Peri',
 'Inhibitory neurons',
 'Oligodendrocytes',
 'T cells',
 'B cells',
 'Microglia']

In [19]:
adata.obs.diagnosis.unique()
adata.obs['condition'] = adata.obs.diagnosis.map(dict(zip(['CTR', 'SPMS', 'PPMS', 'RRMS'],['CNTRL','MS','MS','MS'])))

In [23]:
import scanpy as sc
adata_sub = adata[adata.obs.type_broad.str.contains('Astrocytes')]
# Run differential expression between conditions
sc.tl.rank_genes_groups(
    adata_sub,
    groupby="condition",       # column in adata.obs
    reference="CNTRL",       # baseline group
    method="wilcoxon",         # same as before (or "t-test", "logreg")
    use_raw=False              # set True if you want raw counts/log1p
)

# Extract results for "MS" vs "Control"
deg_condition = sc.get.rank_genes_groups_df(adata_sub, group="MS")
deg_condition.head()
print(deg_condition[deg_condition.names.isin(human_genes)].sort_values(by = 'logfoldchanges', ascending = False))

/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


          names     scores  logfoldchanges         pvals     pvals_adj
3409       LDHA   9.124814        1.002960  7.185957e-20  6.665324e-19
14749      LDHB   0.586018        0.541713  5.578633e-01  1.000000e+00
4145    SLC16A1   7.941396        0.511793  1.999179e-15  1.529336e-14
1322       OPA1  14.615860        0.398638  2.225275e-48  5.120244e-47
2939       MFN1  10.049322        0.392631  9.250188e-24  9.909789e-23
7480       PDK1   4.102162        0.329133  4.093065e-05  1.753750e-04
2321      HIF1A  11.429534        0.273102  2.977008e-30  3.995123e-29
10166   SLC16A3   2.232712        0.214037  2.556797e-02  8.069853e-02
4746        PKM   7.050085        0.108199  1.788086e-12  1.197904e-11
5954      SIRT2   5.566995        0.017592  2.591696e-08  1.389128e-07
8040       MFN2   3.602909       -0.027580  3.146761e-04  1.254970e-03
33054       HK2  -0.606432       -0.370763  5.442279e-01  1.000000e+00
33516      PFKL  -4.510626       -0.530887  6.463658e-06  2.958065e-05
33729 

In [24]:
import scanpy as sc
adata_sub = adata[adata.obs.type_broad.str.contains('Oligodendrocytes')]
# Run differential expression between conditions
sc.tl.rank_genes_groups(
    adata_sub,
    groupby="condition",       # column in adata.obs
    reference="CNTRL",       # baseline group
    method="wilcoxon",         # same as before (or "t-test", "logreg")
    use_raw=False              # set True if you want raw counts/log1p
)

# Extract results for "MS" vs "Control"
deg_condition = sc.get.rank_genes_groups_df(adata_sub, group="MS")
deg_condition.head()
print(deg_condition[deg_condition.names.isin(human_genes)].sort_values(by = 'logfoldchanges', ascending = False))

/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


          names     scores  logfoldchanges          pvals      pvals_adj
16207  PPARGC1A  -0.092444        0.224129   9.263449e-01   1.000000e+00
25912      MFN2  -3.779144        0.062328   1.573685e-04   6.386382e-04
19337      LDHB  -0.327612        0.035229   7.432053e-01   1.000000e+00
31159      OPA1 -15.471097       -0.102832   5.437285e-54   6.450053e-53
31733     SIRT2 -18.227844       -0.183122   3.103183e-74   4.637557e-73
33043      MFN1 -28.600920       -0.381276  6.543996e-180  2.395865e-178
28047      LDHA  -6.777135       -0.420520   1.225823e-11   6.815728e-11
31951     HIF1A -19.399189       -0.421923   7.839741e-84   1.302364e-82
33203      PFKL -30.897209       -0.509926  1.302149e-209  5.807309e-208
31732       PKM -18.221735       -0.540666   3.469820e-74   5.183196e-73
29508      PDK1 -10.041210       -0.649422   1.004342e-23   7.432703e-23
27616   SLC16A3  -6.061020       -0.793436   1.352611e-09   7.007519e-09
32633   SLC16A1 -24.242714       -0.893060  7.89208

In [25]:
import scanpy as sc
adata_sub = adata[adata.obs.type_broad.str.contains('Microglia')]
# Run differential expression between conditions
sc.tl.rank_genes_groups(
    adata_sub,
    groupby="condition",       # column in adata.obs
    reference="CNTRL",       # baseline group
    method="wilcoxon",         # same as before (or "t-test", "logreg")
    use_raw=False              # set True if you want raw counts/log1p
)

# Extract results for "MS" vs "Control"
deg_condition = sc.get.rank_genes_groups_df(adata_sub, group="MS")
deg_condition.head()
print(deg_condition[deg_condition.names.isin(human_genes)].sort_values(by = 'logfoldchanges', ascending = False))

/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/christoffer/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


          names     scores  logfoldchanges         pvals     pvals_adj
7437   PPARGC1A   0.096112        1.749210  9.234318e-01  1.000000e+00
124         PKM  12.274968        0.729479  1.234409e-34  9.697823e-33
32        HIF1A  20.588469        0.696518  3.481999e-94  7.205828e-92
467        LDHA   5.338110        0.459264  9.392026e-08  2.027710e-06
347     SLC16A3   6.680338        0.382859  2.383913e-11  6.986842e-10
751        PFKL   3.594485        0.369085  3.250338e-04  4.652604e-03
2565       OPA1   0.895617        0.069681  3.704575e-01  1.000000e+00
27952      MFN2  -0.345343        0.013924  7.298362e-01  1.000000e+00
10529     SIRT2   0.031906       -0.032993  9.745467e-01  1.000000e+00
29701      PDK1  -0.842740       -0.070522  3.993742e-01  1.000000e+00
31252      MFN1  -1.913362       -0.170361  5.570175e-02  4.623286e-01
33245       HK2  -7.546397       -0.314346  4.474634e-14  1.526277e-12
31586   SLC16A1  -2.282838       -0.652637  2.243992e-02  2.145319e-01
27089 